# Decoder analysis

Before running this notebook, the `decoder.slurm` jobs needs to be run and the data need to be collated into a single csv file:

`awk 'FNR==1 && NR!=1{next;}{print}' build/*_model.csv > build/decoder_predictions.csv`

In [ ]:
import <- function(pkg) { library(pkg, warn.conflicts=F, quietly=T, character.only=T) }
import("repr")
import("stringr")
import("tidyr")
import("dplyr")
import("ggplot2")
import("lme4")
import("emmeans")

In [ ]:
options(repr.matrix.max.cols=15, repr.matrix.max.rows=20)
my.theme <- theme(legend.text=element_text(size=6),
                  legend.title=element_text(size=6),
                  plot.title = element_text(size=7, hjust=0.5),
                  axis.line=element_line(linewidth=0.25),
                  axis.ticks=element_line(linewidth=0.25),
                  axis.ticks.length=unit(0.05, "cm"),
                  axis.title=element_text(size=7),
                  axis.text=element_text(size=6),
                  strip.placement="outside",
                  strip.text=element_text(size=7),
                  strip.background=element_blank())
no.legend <- theme(legend.position="none")
update_geom_defaults("point", list(fill="white", shape=21, size=0.8))
update_geom_defaults("line", list(linewidth=0.4))

In [ ]:
predictions <- (
    data.table::fread("../build/decoder_predictions.csv")
    # shim to fix a typo in the column names
    # |> mutate(score_pred_clean=coalesce(score_pred_clean, socre_pred_clean))
    # |> select(!socre_pred_clean)
)

In [ ]:
predictions |> filter(n_units==63, motif=="9ex2k0dy", background_dBFS==-100) # |> group_by(dataset) |> summarize(score=median(score_actual))

In [ ]:
predictions |> xtabs(~ motif, data=_)

In [ ]:
(
    predictions
    |> filter(background_dBFS==-100)
    |> ggplot(aes(n_units, score_actual, color=dataset))
    + facet_wrap(~ motif)
    + stat_smooth()
    + scale_x_log10()
)

In [ ]:
options(repr.plot.width=2.2, repr.plot.height=2.2, repr.plot.res = 450)
p <- (
    predictions
    |> filter(background_dBFS==-100)
    |> group_by(dataset, n_units, seed)
    |> summarize(score=mean(score_actual))
    |> summarize(y=median(score), ymin=quantile(score, 0.25), ymax=quantile(score, 0.75))
    |> ggplot(aes(n_units, y, color=dataset))
    + geom_point(position=position_dodge(width=0.051), size=1.5)
    + geom_linerange(aes(ymin=ymin, ymax=ymax), position=position_dodge(width=0.05))
    + scale_x_log10("Ensemble size")
    + scale_y_continuous("Prediction score (adj R^2)")
    + theme_classic() + my.theme + no.legend
)
p

In [ ]:
pdf("../figures/decoder_accuracy.pdf", width=2.1, height=2.1)
print(p)
dev.off()

In [ ]:
pred_model <- function(df) {
    wilcox.test(score_actual ~ dataset, df)
}

fm <- (
    predictions
    |> filter(background_dBFS==-100)
    |> filter(n_units < 1000)
    |> group_by(n_units)
    |> nest()
    |> transmute(mdl=purrr::map(data, pred_model))
    |> mutate(stats=purrr::map(mdl, broom::tidy))
    |> select(n_units, stats)
    |> unnest(cols=stats)
    |> arrange(n_units)
)
fm

In [ ]:
fm <- (
    predictions
    |> filter(background_dBFS==-100)
    |> filter(n_units < 1000)
    |> mutate(n_units=factor(n_units))
    |> lmer(score_actual ~ dataset*n_units + (1|motif), data=_)
)

In [ ]:
joint_tests(fm)

## Noise invariance

Using the response to noisy stimuli as input, how similar are the predictions to the predicted stimulus from the clean stimuli?

First, just use all the units. In principle, the difference in decoding performance should not matter because we're comparing
to the decoded stimulus, not the original. Calculating standard error across motifs.

In [ ]:
options(repr.plot.width=1.9, repr.plot.height=1.4, repr.plot.res = 450)
p <- (
    predictions
    |> filter(background_dBFS != -100, (dataset=="cr_units" & n_units==1778) | (dataset=="pr_units" & n_units==927))
    |> group_by(dataset, background_dBFS)
    # typo:
    |> summarize(y=mean(score_pred_clean), yse=sd(score_pred_clean)/sqrt(n()))
    |> ggplot(aes(-30 - background_dBFS, y, color=dataset))
    + geom_point(position=position_dodge(width=0.051), size=1.5)
    + geom_linerange(aes(ymin=y-yse, ymax=y+yse), position=position_dodge(width=0.05))
    + scale_x_reverse("SNR (dB)")
    + scale_y_continuous("Similarity to 70 dB SNR (R^2)")
    + theme_classic() + my.theme + no.legend
)
p

In [ ]:
pdf("../figures/decoder_invariance_all_units.pdf", width=1.9, height=1.4)
print(p)
dev.off()

In [ ]:
fm <- (
    predictions
    |> filter(background_dBFS != -100, (dataset=="cr_units" & n_units==1778) | (dataset=="pr_units" & n_units==927))
    |> mutate(background_dBFS=factor(background_dBFS))
    |> lm(score_pred_clean ~ background_dBFS*dataset, data=_)
)
joint_tests(fm)

In [ ]:
(
    fm
    |> emmeans(~ dataset) |> contrast("pairwise")
)

Alternatively, compare ensembles matched in size.

In [ ]:
options(repr.plot.width=2.2, repr.plot.height=3.5, repr.plot.res = 450)
p <- (
    predictions
    |> filter(background_dBFS != -100, n_units %in% c(63, 154, 741))
    |> group_by(n_units, dataset, background_dBFS, seed)
    |> summarize(score=mean(score_pred_clean))
    |> summarize(y=median(score), ymin=quantile(score, 0.25), ymax=quantile(score, 0.75))
    |> ggplot(aes(-30 - background_dBFS, y, color=dataset))
    + facet_grid(rows = vars(n_units))
    + geom_point(size=1.5)
    + geom_linerange(aes(ymin=ymin, ymax=ymax))
    + scale_x_reverse("SNR (dB)")
    + scale_y_continuous("Similarity to 70 dB SNR (R^2)")
    + theme_classic() + my.theme + no.legend
)
p

Can also average across replicates and use motifs for standard errors

In [ ]:
options(repr.plot.width=2.2, repr.plot.height=3.5, repr.plot.res = 450)
p <- (
    predictions
    |> filter(background_dBFS != -100, n_units %in% c(63, 473, 927))
    |> group_by(n_units, dataset, background_dBFS, motif)
    |> summarize(score=mean(score_pred_clean))
    |> summarize(y=mean(score), yse=sd(score)/sqrt(n()))
    |> ggplot(aes(-30 - background_dBFS, y, color=dataset))
    + facet_grid(rows = vars(n_units))
    + geom_point(size=1.5)
    + geom_linerange(aes(ymin=y-yse, ymax=y+yse))
    + scale_x_reverse("SNR (dB)")
    + scale_y_continuous("Similarity to 70 dB SNR (R^2)")
    + theme_classic() + my.theme + no.legend
)
p

But actually the simplest is to just compare to the actual stimulus and let readers judge how the curves differ in shape.

In [ ]:
options(repr.plot.width=2.2, repr.plot.height=3.5, repr.plot.res = 450)
p <- (
    predictions
    |> filter(n_units %in% c(63, 473, 927))
    |> group_by(n_units, dataset, background_dBFS, seed)
    |> summarize(score=mean(score_actual))
    #|> summarize(y=mean(score), yse=sd(score)/sqrt(n()), ymin=y-yse, ymax=y+yse)
    |> summarize(y=median(score), ymin=quantile(score, 0.25), ymax=quantile(score, 0.75))
    |> ggplot(aes(-30 - background_dBFS, y, color=dataset))
    + facet_grid(rows = vars(n_units))
    + geom_point(size=1.5)
    + geom_linerange(aes(ymin=ymin, ymax=ymax))
    + scale_x_reverse("SNR (dB)")
    + scale_y_continuous("Prediction Score (R^2)")
    + theme_classic() + my.theme + no.legend
)
p

In [ ]:
pdf("../figures/decoder_invariance_ensembles.pdf", width=2.2, height=3.5)
print(p)
dev.off()